In [2]:
! pip install sodapy

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
# Import dependencies
import requests
import pandas as pd
from pprint import pprint
from sodapy import Socrata
import datetime

In [6]:
# Retrieve data using Socrata function provided by sodapy (API developers)
client = Socrata("data.cityofchicago.org", None)
results = client.get("6zsd-86xi", year=2018, limit=500000)

# Convert  results to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df.head()

,arrest,beat,block,case_number,community_area,date,description,district,domestic,fbi_code,...,latitude,location,location_description,longitude,primary_type,updated_on,ward,x_coordinate,y_coordinate,year
0,False,0524,117XX S SANGAMON ST,JB140548,53,2018-02-04T01:36:00.000,FIRST DEGREE MURDER,005,False,01A,...,41.68073915,"{'type': 'Point', 'coordinates': [-87.64566114...",STREET,-87.645661144,HOMICIDE,2018-02-11T15:57:41.000,34,1172100,1827003,2018
1,False,2234,011XX W 111TH ST,JB107989,75,2018-01-07T21:01:00.000,DOMESTIC BATTERY SIMPLE,022,True,08B,...,41.692258729,"{'type': 'Point', 'coordinates': [-87.65053259...",RESIDENCE,-87.65053259,BATTERY,2018-05-04T15:51:04.000,34,1170736,1831190,2018
2,False,1834,004XX N MICHIGAN AVE,JB108319,8,2018-01-05T18:10:00.000,RETAIL THEFT,018,False,06,...,41.890384236,"{'type': 'Point', 'coordinates': [-87.62410991...",DEPARTMENT STORE,-87.624109917,THEFT,2018-05-04T15:51:04.000,42,1177357,1903444,2018
3,True,1014,016XX S HARDING AVE,JB108320,29,2018-01-08T08:59:00.000,MANU/DELIVER: HEROIN (WHITE),010,False,18,...,41.858283017,"{'type': 'Point', 'coordinates': [-87.72369127...",RESIDENCE,-87.723691271,NARCOTICS,2018-05-04T15:51:04.000,24,1150326,1891538,2018
4,False,0222,047XX S LAKE PARK AVE,JB108379,39,2018-01-08T10:45:00.000,HARASSMENT BY TELEPHONE,002,True,26,...,41.809297059,"{'type': 'Point', 'coordinates': [-87.59196702...",APARTMENT,-87.591967028,OTHER OFFENSE,2018-05-04T15:51:04.000,4,1186364,1873970,2018


In [10]:
# Extract useful rows into 'df' which will be saved to csv after munging
df = results_df[['case_number', 'beat', 'community_area', 'ward', 'date', 'year', 'description', 'primary_type', 
                 'fbi_code', 'latitude', 'longitude', 'domestic', 'arrest']]

df = df.dropna()

# Parse datetime into date and time.
df[['date_parsed', 'time_parsed']] = df['date'].str.split('T', expand=True)
df = df.drop(columns=['date', 'year'])
df = df.rename(columns = {'date_parsed': 'date', 'time_parsed': 'time'})
df.head()

df['month'] = pd.DatetimeIndex(df['date']).month
df['year'] = pd.DatetimeIndex(df['date']).year
df['day'] = pd.DatetimeIndex(df['date']).day
df['hour'] = pd.DatetimeIndex(df['time']).hour
df.head()

len(df)

260400

In [30]:
#Save output to csv
df.to_csv('2018_crime.csv', index=False)